<a href="https://colab.research.google.com/github/KIMMOOKYONG/COLAB-NOTENOOK/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%ED%85%8C%EB%A7%88%EB%B3%84%EC%A2%85%EB%AA%A9%EC%A0%95%EB%B3%B4%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import requests
from io import BytesIO
import pandas as pd
import seaborn as sns
from tqdm import tqdm
 
from bs4 import BeautifulSoup
import datetime
import re

In [5]:
#%% 
end = datetime.datetime.now()
start = end - datetime.timedelta(days=10)
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', \
           "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", \
           "Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
pd.set_option('display.float_format', None)
df = pd.DataFrame()

In [6]:
# 테마별 종목정보 수집
# 페이징 번호
for pagenum in tqdm(range(1,8)):
    url = "https://finance.naver.com/sise/theme.nhn?field=name&ordering=asc&page={pagenum}".format(pagenum=pagenum)
    resp = requests.get(url, headers = headers)
    soup = BeautifulSoup(resp.content, "html.parser")

    thema_num = 0
    skip_head_num = 3
    skip_blank = 3
    while thema_num < 10:
        try : 
            thema_num += 1
            board_date = soup.select("#contentarea_left > table.type_1.theme > tr:nth-of-type("+str(skip_head_num + thema_num)+") > td.col_type1 > a")[0]
            # <a href="/sise/sise_group_detail.naver?type=theme&amp;no=414">2019 상반기 신규상장</a>
            linkUrl = 'https://finance.naver.com' + board_date['href']
            # https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=414
 
            linkResp = requests.get(linkUrl, headers = headers)            
            linkSoup = BeautifulSoup(linkResp.content, "html.parser")
            thema = board_date.text
            # 2019 상반기 신규상장
        except Exception as e:
            continue

        # 테마별 종목
        try : 
            data = pd.read_html(linkUrl, encoding="euc-kr")[2] \
                           .dropna(how="all")[["종목명", "현재가","전일비","등락률","매수호가","매도호가","거래량","거래대금","전일거래량"]]
            data["테마"] = thema

            df = df.append(data, ignore_index =True)            
        except : 
            continue

 86%|████████▌ | 6/7 [02:54<00:28, 28.51s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 7/7 [03:20<00:00, 28.65s/it]


In [7]:
df

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,테마
0,마이크로디지탈 *,5040.0,360.0,+7.69%,5060.0,5080.0,135994.0,693.0,100346.0,2019 상반기 신규상장
1,까스텔바작 *,8840.0,410.0,+4.86%,8770.0,8810.0,244279.0,2151.0,329685.0,2019 상반기 신규상장
2,셀리드 *,18900.0,800.0,+4.42%,18850.0,19000.0,73019.0,1380.0,76716.0,2019 상반기 신규상장
3,코퍼스코리아 *,2270.0,35.0,+1.57%,2280.0,2290.0,421717.0,960.0,450681.0,2019 상반기 신규상장
4,지노믹트리 *,13950.0,200.0,+1.45%,13900.0,13950.0,55496.0,769.0,74361.0,2019 상반기 신규상장
...,...,...,...,...,...,...,...,...,...,...
1848,일진파워 *,15850.0,200.0,-1.25%,15900.0,15950.0,372398.0,6051.0,482278.0,핵융합에너지
1849,다원시스 *,22700.0,300.0,-1.30%,22750.0,22800.0,113275.0,2586.0,391837.0,핵융합에너지
1850,고려제강,20500.0,500.0,-2.38%,20550.0,20650.0,11129.0,229.0,30036.0,핵융합에너지
1851,두산에너빌리티,17750.0,650.0,-3.53%,17900.0,17950.0,3731383.0,67879.0,5072898.0,핵융합에너지


In [8]:
df.to_csv("테마별종목정보.csv")

In [11]:
df1 = pd.read_csv("테마별종목정보.csv")
del df1["Unnamed: 0"]
df1

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,테마
0,마이크로디지탈 *,5040.0,360.0,+7.69%,5060.0,5080.0,135994.0,693.0,100346.0,2019 상반기 신규상장
1,까스텔바작 *,8840.0,410.0,+4.86%,8770.0,8810.0,244279.0,2151.0,329685.0,2019 상반기 신규상장
2,셀리드 *,18900.0,800.0,+4.42%,18850.0,19000.0,73019.0,1380.0,76716.0,2019 상반기 신규상장
3,코퍼스코리아 *,2270.0,35.0,+1.57%,2280.0,2290.0,421717.0,960.0,450681.0,2019 상반기 신규상장
4,지노믹트리 *,13950.0,200.0,+1.45%,13900.0,13950.0,55496.0,769.0,74361.0,2019 상반기 신규상장
...,...,...,...,...,...,...,...,...,...,...
1848,일진파워 *,15850.0,200.0,-1.25%,15900.0,15950.0,372398.0,6051.0,482278.0,핵융합에너지
1849,다원시스 *,22700.0,300.0,-1.30%,22750.0,22800.0,113275.0,2586.0,391837.0,핵융합에너지
1850,고려제강,20500.0,500.0,-2.38%,20550.0,20650.0,11129.0,229.0,30036.0,핵융합에너지
1851,두산에너빌리티,17750.0,650.0,-3.53%,17900.0,17950.0,3731383.0,67879.0,5072898.0,핵융합에너지


In [16]:
df1["테마"].unique()

array(['2019 상반기 신규상장', '2019 하반기 신규상장', '2020 상반기 신규상장', '2020 하반기 신규상장',
       '2021 상반기 신규상장', '2021 하반기 신규상장', '2022 상반기 신규상장', '2022 하반기 신규상장',
       '2차전지', '2차전지(LFP/리튬인산철)', 'OLED(유기 발광 다이오드)', 'PCB(FPCB 등)',
       'RFID(NFC 등)', 'SI(시스템통합)', 'SNS(소셜네트워크서비스)', 'SSD',
       'U-Healthcare(원격진료)', 'UAM(도심항공모빌리티)', '가상현실(VR)', '가상화폐(비트코인 등)',
       '마리화나(대마)', '마스크', '마이데이터', '마이크로 LED', '마이크로바이옴', '마켓컬리 관련주',
       '메르스 코로나 바이러스', '메타버스(Metaverse)', '면세점', '면역항암제', '소매유통',
       '소모성자재구매대행(MRO)', '손해보험', '수산', '수소차(연료전지/부품/충전소 등)',
       '수자원(양적/질적 개선)', '슈퍼박테리아', '스마트그리드(지능형전력망)', '스마트카(SMART CAR)',
       '스마트팩토리(스마트공장)', '인터넷 대표주', '인터넷은행', '일자리(취업)', '자동차 대표주', '자동차부품',
       '자원개발', '자율주행차', '자전거', '재난/안전(지진 등)', '재택근무/스마트워크', '출산장려정책',
       '치매', '치아 치료(임플란트 등)', '카메라모듈/부품', '카지노', '카카오뱅크 관련주', '캐릭터상품',
       '케이블TV SO/MSO', '코로나19(나파모스타트)', '코로나19(덱사메타손)', '폐배터리 관련주',
       '폴더블폰', '풍력에너지', '플렉서블 디스플레이', '핀테크(FinTech)', '항공/저가 항공사(LCC)',
       '항공기부품', '해운', '

In [ ]:
# 테마 정보 수집
# 페이징 번호
for pagenum in tqdm(range(1,8)):
    url = "https://finance.naver.com/sise/theme.nhn?field=name&ordering=asc&page={pagenum}".format(pagenum=pagenum)
    resp = requests.get(url, headers = headers)
    soup = BeautifulSoup(resp.content, "html.parser")

    thema_num = 0
    skip_head_num = 3
    skip_blank = 3
    while thema_num < 10:
        try : 
            thema_num += 1
            board_date = soup.select("#contentarea_left > table.type_1.theme > tr:nth-of-type("+str(skip_head_num + thema_num)+") > td.col_type1 > a")[0]
            # <a href="/sise/sise_group_detail.naver?type=theme&amp;no=414">2019 상반기 신규상장</a>
            linkUrl = 'https://finance.naver.com' + board_date['href']
            # https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=414
 
            linkResp = requests.get(linkUrl, headers = headers)            
            linkSoup = BeautifulSoup(linkResp.content, "html.parser")
            thema = board_date.text
            # 2019 상반기 신규상장
        except Exception as e:
            continue

        # 테마별 종목
        try : 
            data = pd.read_html(linkUrl, encoding="euc-kr")[2] \
                           .dropna(how="all")[["종목명", "현재가","전일비","등락률","매수호가","매도호가","거래량","거래대금","전일거래량"]]
            data["테마"] = thema

            df = df.append(data, ignore_index =True)            
        except : 
            continue

In [ ]:
# 업종별 종목별 정보 수집
# 페이징 번호
for pagenum in tqdm(range(1,8)):
    url = "https://finance.naver.com/sise/theme.nhn?field=name&ordering=asc&page={pagenum}".format(pagenum=pagenum)
    resp = requests.get(url, headers = headers)
    soup = BeautifulSoup(resp.content, "html.parser")

    thema_num = 0
    skip_head_num = 3
    skip_blank = 3
    while thema_num < 10:
        try : 
            thema_num += 1
            board_date = soup.select("#contentarea_left > table.type_1.theme > tr:nth-of-type("+str(skip_head_num + thema_num)+") > td.col_type1 > a")[0]
            # <a href="/sise/sise_group_detail.naver?type=theme&amp;no=414">2019 상반기 신규상장</a>
            linkUrl = 'https://finance.naver.com' + board_date['href']
            # https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=414
 
            linkResp = requests.get(linkUrl, headers = headers)            
            linkSoup = BeautifulSoup(linkResp.content, "html.parser")
            thema = board_date.text
            # 2019 상반기 신규상장
        except Exception as e:
            continue

        # 테마별 종목
        try : 
            data = pd.read_html(linkUrl, encoding="euc-kr")[2] \
                           .dropna(how="all")[["종목명", "현재가","전일비","등락률","매수호가","매도호가","거래량","거래대금","전일거래량"]]
            data["테마"] = thema

            df = df.append(data, ignore_index =True)            
        except : 
            continue

In [ ]:
# 업종 정보 수집
# 페이징 번호
for pagenum in tqdm(range(1,8)):
    url = "https://finance.naver.com/sise/theme.nhn?field=name&ordering=asc&page={pagenum}".format(pagenum=pagenum)
    resp = requests.get(url, headers = headers)
    soup = BeautifulSoup(resp.content, "html.parser")

    thema_num = 0
    skip_head_num = 3
    skip_blank = 3
    while thema_num < 10:
        try : 
            thema_num += 1
            board_date = soup.select("#contentarea_left > table.type_1.theme > tr:nth-of-type("+str(skip_head_num + thema_num)+") > td.col_type1 > a")[0]
            # <a href="/sise/sise_group_detail.naver?type=theme&amp;no=414">2019 상반기 신규상장</a>
            linkUrl = 'https://finance.naver.com' + board_date['href']
            # https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=414
 
            linkResp = requests.get(linkUrl, headers = headers)            
            linkSoup = BeautifulSoup(linkResp.content, "html.parser")
            thema = board_date.text
            # 2019 상반기 신규상장
        except Exception as e:
            continue

        # 테마별 종목
        try : 
            data = pd.read_html(linkUrl, encoding="euc-kr")[2] \
                           .dropna(how="all")[["종목명", "현재가","전일비","등락률","매수호가","매도호가","거래량","거래대금","전일거래량"]]
            data["테마"] = thema

            df = df.append(data, ignore_index =True)            
        except : 
            continue

In [41]:
df2 = pd.read_html("https://finance.naver.com/sise/sise_group.naver?type=upjong", encoding="euc-kr")[0]\
                .dropna(how="all")
# 멀티헤더 칼럼명을 단일헤드 칼럼명으로 변환하기
df2.columns = df2.columns.get_level_values(0)
df2

,업종명,전일대비,전일대비 등락현황,전일대비 등락현황,전일대비 등락현황,전일대비 등락현황,등락그래프
1,건강관리기술,+4.79%,11.0,8.0,1.0,2.0,69%
2,출판,+4.61%,4.0,3.0,1.0,0.0,66%
3,디스플레이패널,+3.66%,4.0,3.0,0.0,1.0,53%
4,판매업체,+2.98%,12.0,6.0,3.0,3.0,43%
5,소프트웨어,+2.84%,58.0,43.0,4.0,11.0,41%
...,...,...,...,...,...,...,...
117,에너지장비및서비스,-4.99%,21.0,3.0,1.0,17.0,72%
121,석유와가스,-5.16%,20.0,3.0,0.0,17.0,74%
122,전기장비,-5.69%,28.0,7.0,2.0,19.0,82%
123,무역회사와판매업체,-6.05%,5.0,1.0,0.0,4.0,87%


In [43]:
df3 = pd.read_html("https://finance.naver.com/sise/sise_group.naver?type=upjong", encoding="euc-kr")[0]\
                .dropna(how="all")
# 멀티헤더 칼럼명을 단일헤드 칼럼명으로 변환하기
df3.columns = df3.columns.get_level_values(1)
df3

,업종명,전일대비,전체,상승,보합,하락,등락그래프
1,건강관리기술,+4.79%,11.0,8.0,1.0,2.0,69%
2,출판,+4.61%,4.0,3.0,1.0,0.0,66%
3,디스플레이패널,+3.66%,4.0,3.0,0.0,1.0,53%
4,판매업체,+2.98%,12.0,6.0,3.0,3.0,43%
5,소프트웨어,+2.84%,58.0,43.0,4.0,11.0,41%
...,...,...,...,...,...,...,...
117,에너지장비및서비스,-4.99%,21.0,3.0,1.0,17.0,72%
121,석유와가스,-5.16%,20.0,3.0,0.0,17.0,74%
122,전기장비,-5.69%,28.0,7.0,2.0,19.0,82%
123,무역회사와판매업체,-6.05%,5.0,1.0,0.0,4.0,87%


In [62]:
pd.read_html("https://finance.naver.com/item/main.naver?code=356890", encoding="euc-kr")[9]

,0,1
0,동일업종 PER,18.35배
1,동일업종 등락률,+0.95%
